In [1]:
# tf-idf for creative_id and advertiser_id
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

train_merged=pd.read_csv('train_merged.csv')
v = TfidfVectorizer(max_df=30,lowercase=False,tokenizer=lambda x:x)

creative_id=train_merged.groupby("user_id")["creative_id"].apply(list)
creative_id = v.fit_transform(creative_id)
advertiser_id=train_merged.groupby("user_id")["advertiser_id"].apply(list)
advertiser_id = v.fit_transform(advertiser_id)
cr_ad=sparse.hstack((creative_id,advertiser_id),dtype=float)
display(cr_ad)
sparse.save_npz('cr_ad.npz', cr_ad)

In [1]:
# predict gender
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse

user_info=pd.read_csv('user_info.csv')
creative_matrix=sparse.load_npz('cr_ad.npz')
X_train=creative_matrix
y_train=user_info["gender"]
display(X_train,y_train)
train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('gender.bin')

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


<900000x2451045 sparse matrix of type '<class 'numpy.float64'>'
	with 10312696 stored elements in COOrdinate format>

0         0
1         0
2         1
3         0
4         0
         ..
899995    0
899996    1
899997    1
899998    0
899999    1
Name: gender, Length: 900000, dtype: int64

In [ ]:
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd

train_data = lgb.Dataset('gender.bin')
lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
print("start cv")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, shuffle=True,
    verbose_eval=1, show_stdv=True)

start cv


# using test data to train

In [ ]:
# predict gender
# tf-idf for creative_id
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

predict_merged=pd.read_csv('predict_merged.csv')
v = TfidfVectorizer(max_df=30,lowercase=False,tokenizer=lambda x:x)

creative_id=predict_merged.groupby("user_id")["creative_id"].apply(list)
creative_id = v.fit_transform(creative_id)
advertiser_id=predict_merged.groupby("user_id")["advertiser_id"].apply(list)
advertiser_id = v.fit_transform(advertiser_id)
pred_cr_ad=sparse.hstack((creative_id,advertiser_id),dtype=float)
display(pred_cr_ad)
sparse.save_npz('pred_cr_ad.npz', pred_cr_ad)

In [ ]:
# predict gender
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse
from sklearn.model_selection import KFold

X_train=sparse.load_npz('cr_ad.npz')
y_train=user_info["gender"]
X_test=sparse.load_npz('pred_cr_ad.npz')

folds = KFold(n_splits=5, shuffle=False)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])
 
    model = lgb.train(param, 
                    trn_data, 
                    num_round=200, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval = 10, 
                    early_stopping_rounds = 100)
    predictions += clf.predict(X_test, num_iteration=model.best_iteration) / folds.n_splits
    display(predictions)

display(predictions)

In [1]:
%reset -f
# !pip install lightgbm
import lightgbm as lgb
train_data = lgb.Dataset('gender.bin')
lgb_params = {'num_leaves': 2**5-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':12,
             }
print("start training")
model = lgb.train(lgb_params, train_data,num_boost_round=120)
model.save_model("gender_model.txt")
print("training completed!")

start training
training completed!


In [25]:
import lightgbm as lgb
from scipy import sparse

X_pred=creative_id_matrix
zero_matrix=sparse.csc_matrix((1000000,2533238-2497542))
X_pred=sparse.hstack([X_pred,zero_matrix],dtype=float).tocsr()
print("start predicting")
model=lgb.Booster(model_file='gender_model.txt')
y_pred=model.predict(X_pred)
y_pred=(y_pred>0.5)*1
display(y_pred)
submission=pd.DataFrame([i for i in range(3000001,4000001)],columns=["user_id"])
submission["predicted_gender"]=y_pred
display(submission)

start predicting


array([0, 0, 0, ..., 0, 0, 0])

,user_id,predicted_gender
0,3000001,0
1,3000002,0
2,3000003,0
3,3000004,0
4,3000005,0
...,...,...
999995,3999996,0
999996,3999997,0
999997,3999998,0
999998,3999999,0


In [30]:
submission["predicted_age"]=[1 for _ in range(int(1e6))]

In [37]:
submission["predicted_gender"]+=1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype
---  ------            --------------    -----
 0   user_id           1000000 non-null  int64
 1   predicted_gender  1000000 non-null  int64
 2   predicted_age     1000000 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB


In [38]:
submission.to_csv("submission.csv",index=False)